In [1]:
import os                          # operating system interfaces
import glob                        # working with OS pathnames
import time                        # time processing
import numpy as np                 # linear algebra
import pandas as pd                # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt    # data visualization purposes
import seaborn as sns     
import joblib         # statistical data visualization
sns.set_theme()

In [2]:
# Constants
DATASET_SRC = "time_series_aq"

# Data-Preprocessing

1. Feature Reduction
2. Missing Values

In [3]:
df_states = pd.read_csv(f'{DATASET_SRC}/stations_info.csv')
df_states

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year
0,AP001,Andhra Pradesh,Tirupati,APPCB,"Tirumala, Tirupati",July,7,2016
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017
4,AP005,Andhra Pradesh,Amaravati,APPCB,"Secretariat, Amaravati",November,11,2017
...,...,...,...,...,...,...,...,...
448,WB010,West Bengal,Howrah,WBPCB,"Belur Math, Howrah",May,5,2019
449,WB011,West Bengal,Kolkata,WBPCB,"Jadavpur, Kolkata",June,6,2019
450,WB012,West Bengal,Kolkata,WBPCB,"Rabindra Sarobar, Kolkata",August,8,2019
451,WB013,West Bengal,Kolkata,WBPCB,"Bidhannagar, Kolkata",August,8,2019


In [4]:
df_states.drop(columns=['agency', 'station_location', 'start_month'], inplace=True)

In [5]:
def combine_state_df(state_name):
    '''
    Combine all state files into a single dataframe and attaching the city information.

    Parameters
    ----------
        state_name (str): The name of the state

    Return
    ------
        df (DataFrame): The combined dataframe from all files of a specific state
    '''

    state_code = df_states[df_states['state'] == state_name]['file_name'].iloc[0][:2]
    state_files = glob.glob(f'{DATASET_SRC}/{state_code}*.csv')
    print(f'Combining a total of {len(state_files)} files...\n')

    combined_df = []

    for state_file in state_files:
        print(f"state_file: {state_file}")
        # file_name = state_file.split(f'{DATASET_SRC}/')[1][0:-4]
        file_name = os.path.basename(state_file)[:-4]
        print(f"file_name: {file_name}")
        file_df = pd.read_csv(state_file)
        file_df['city'] = df_states[df_states['file_name'] == file_name]['city'].values[0]
        file_df['city'] = file_df['city'].astype('string')
        combined_df.append(file_df)

    return pd.concat(combined_df)

In [6]:
df = combine_state_df('Delhi')

Combining a total of 40 files...

state_file: time_series_aq/DL040.csv
file_name: DL040
state_file: time_series_aq/DL034.csv
file_name: DL034
state_file: time_series_aq/DL020.csv
file_name: DL020
state_file: time_series_aq/DL008.csv
file_name: DL008
state_file: time_series_aq/DL009.csv
file_name: DL009
state_file: time_series_aq/DL021.csv
file_name: DL021
state_file: time_series_aq/DL035.csv
file_name: DL035
state_file: time_series_aq/DL023.csv
file_name: DL023
state_file: time_series_aq/DL037.csv
file_name: DL037
state_file: time_series_aq/DL036.csv
file_name: DL036
state_file: time_series_aq/DL022.csv
file_name: DL022
state_file: time_series_aq/DL026.csv
file_name: DL026
state_file: time_series_aq/DL032.csv
file_name: DL032
state_file: time_series_aq/DL033.csv
file_name: DL033
state_file: time_series_aq/DL027.csv
file_name: DL027
state_file: time_series_aq/DL019.csv
file_name: DL019
state_file: time_series_aq/DL031.csv
file_name: DL031
state_file: time_series_aq/DL025.csv
file_name: 

In [7]:
df.head()

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),...,O Xylene (ug/m3),CH4 (ug/m3),NMHC (ug/m3),SPM (ug/m3),THC (ug/m3),Gust (km/hr),MP-Xylene (),Eth-Benzene (),Xylene (),CH4 ()
0,2020-11-13 14:00:00,2020-11-13 15:00:00,142.70,390.00,0.57,17.92,18.48,48.87,2.78,0.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-13 15:00:00,2020-11-13 16:00:00,130.43,326.85,1.58,17.61,19.19,56.64,2.69,0.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-13 16:00:00,2020-11-13 17:00:00,114.45,274.28,3.96,24.29,28.26,86.66,2.45,0.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-13 17:00:00,2020-11-13 18:00:00,148.47,292.00,14.84,37.68,52.52,114.45,2.50,1.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-13 18:00:00,2020-11-13 19:00:00,151.22,449.30,40.67,46.61,87.29,106.35,3.21,3.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
del df['From Date']
del df['To Date']

In [9]:
df.head()

,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),Ozone (ug/m3),Benzene (ug/m3),...,O Xylene (ug/m3),CH4 (ug/m3),NMHC (ug/m3),SPM (ug/m3),THC (ug/m3),Gust (km/hr),MP-Xylene (),Eth-Benzene (),Xylene (),CH4 ()
0,142.70,390.00,0.57,17.92,18.48,48.87,2.78,0.83,39.24,2.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,130.43,326.85,1.58,17.61,19.19,56.64,2.69,0.71,30.93,2.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,114.45,274.28,3.96,24.29,28.26,86.66,2.45,0.84,26.15,2.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148.47,292.00,14.84,37.68,52.52,114.45,2.50,1.65,22.16,3.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,151.22,449.30,40.67,46.61,87.29,106.35,3.21,3.24,21.95,5.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
reduction_groups = {
    "Xylene (ug/m3)":    ["Xylene ()"],
    "MP-Xylene (ug/m3)": ["MP-Xylene ()"],
    "Benzene (ug/m3)":   ["Benzene ()"],
    "Toluene (ug/m3)":   ["Toluene ()"],
    "SO2 (ug/m3)":       ["SO2 ()"],
    "NOx (ug/m3)":       ["NOx (ppb)"],
    "Ozone (ug/m3)":     ["Ozone (ppb)"],
    "AT (degree C)":     ["AT ()"],
    "WD (degree)":       ["WD (degree C)", "WD (deg)", "WD ()"],
    "WS (m/s)":          ["WS ()"]
}

In [11]:
def merge_columns(dataframe, columns):
    '''
    Merges column records into a single column.

    Parameters
    ----------
        dataframe (DataFrame): The DataFrame to edit
        column (str): The name of the column to merge records into
        cols_to_merge (list[str]): A list of column names to retrieve records
    '''

    for column, cols_to_merge in columns.items():
        # Check if the original column exist, otherwise create it
        if column not in dataframe.columns and any(name in dataframe.columns for name in cols_to_merge):
            dataframe[column] = np.nan

        for col_name in cols_to_merge:
            if col_name in dataframe.columns:
                dataframe[column] = dataframe[column].fillna(dataframe[col_name])
                dataframe = dataframe.drop(columns=[col_name])

    return dataframe


In [12]:
df = merge_columns(df, reduction_groups)

In [13]:
df = df.dropna(how='all')
df = df.dropna(how='all', axis='columns')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2796171 entries, 0 to 20841
Data columns (total 36 columns):
 #   Column               Dtype  
---  ------               -----  
 0   PM2.5 (ug/m3)        float64
 1   PM10 (ug/m3)         float64
 2   NO (ug/m3)           float64
 3   NO2 (ug/m3)          float64
 4   NH3 (ug/m3)          float64
 5   SO2 (ug/m3)          float64
 6   CO (mg/m3)           float64
 7   Ozone (ug/m3)        float64
 8   Benzene (ug/m3)      float64
 9   Toluene (ug/m3)      float64
 10  Eth-Benzene (ug/m3)  float64
 11  MP-Xylene (ug/m3)    float64
 12  RH (%)               float64
 13  WS (m/s)             float64
 14  WD (degree)          float64
 15  BP (mmHg)            float64
 16  Xylene (ug/m3)       float64
 17  AT (degree C)        float64
 18  RF (mm)              float64
 19  city                 string 
 20  Temp (degree C)      float64
 21  SR (W/mt2)           float64
 22  CO (ug/m3)           float64
 23  VWS (m/s)            float64
 24  RH ()

In [15]:
del df['city']
del df['RH ()']
del df['CH4 ()']
del df['SR ()']
del df['BP ()']

In [16]:
column_mapping = {
    'PM2.5 (ug/m3)': 'PM2_5',
    'PM10 (ug/m3)': 'PM10',
    'NO (ug/m3)': 'NO',
    'NO2 (ug/m3)': 'NO2',
    'NH3 (ug/m3)': 'NH3',
    'SO2 (ug/m3)': 'SO2',
    'CO (mg/m3)': 'CO_mg',
    'Ozone (ug/m3)': 'Ozone',
    'Benzene (ug/m3)': 'Benzene',
    'Toluene (ug/m3)': 'Toluene',
    'Eth-Benzene (ug/m3)': 'Eth_Benzene',
    'MP-Xylene (ug/m3)': 'MP_Xylene',
    'Temp (degree C)': 'Temp',
    'RH (%)': 'RH',
    'WS (m/s)': 'WS',
    'WD (degree)': 'WD',
    'SR (W/mt2)': 'SR',
    'BP (mmHg)': 'BP',
    'VWS (m/s)': 'VWS',
    'CH4 (ug/m3)': 'CH4',
    'AT (degree C)': 'AT',
    'THC (ug/m3)': 'THC',
    'Xylene (ug/m3)': 'Xylene',
    'RF (mm)': 'RF',
    'CO (ug/m3)': 'CO_ug',
    'O Xylene (ug/m3)': 'O_Xylene',
    'CO (ng/m3)': 'CO_ng',
    'MH (m)': 'MH',
    'HCHO (ug/m3)': 'HCHO',
    'Hg (ug/m3)': 'Hg',
    'NOx (ug/m3)': 'NOx',
}

# Rename the columns using the mapping
df = df.rename(columns=column_mapping)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2796171 entries, 0 to 20841
Data columns (total 31 columns):
 #   Column       Dtype  
---  ------       -----  
 0   PM2_5        float64
 1   PM10         float64
 2   NO           float64
 3   NO2          float64
 4   NH3          float64
 5   SO2          float64
 6   CO_mg        float64
 7   Ozone        float64
 8   Benzene      float64
 9   Toluene      float64
 10  Eth_Benzene  float64
 11  MP_Xylene    float64
 12  RH           float64
 13  WS           float64
 14  WD           float64
 15  BP           float64
 16  Xylene       float64
 17  AT           float64
 18  RF           float64
 19  Temp         float64
 20  SR           float64
 21  CO_ug        float64
 22  VWS          float64
 23  CO_ng        float64
 24  NOx          float64
 25  MH           float64
 26  HCHO         float64
 27  Hg           float64
 28  O_Xylene     float64
 29  CH4          float64
 30  THC          float64
dtypes: float64(31)
memory usage: 68

In [18]:
df.columns

Index(['PM2_5', 'PM10', 'NO', 'NO2', 'NH3', 'SO2', 'CO_mg', 'Ozone', 'Benzene',
       'Toluene', 'Eth_Benzene', 'MP_Xylene', 'RH', 'WS', 'WD', 'BP', 'Xylene',
       'AT', 'RF', 'Temp', 'SR', 'CO_ug', 'VWS', 'CO_ng', 'NOx', 'MH', 'HCHO',
       'Hg', 'O_Xylene', 'CH4', 'THC'],
      dtype='object')

In [19]:
df = df[['PM2_5', 'PM10', 'NO2', 'SO2', 'CO_ug', 'Ozone', 'Temp', 'RH', 'WS', 'BP', 'WD']]

In [20]:
# Convert all columns of dtype float64 to float32
for column in df.select_dtypes(include=['float64']).columns:
    df[column] = df[column].astype('float32')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2796171 entries, 0 to 20841
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   PM2_5   float32
 1   PM10    float32
 2   NO2     float32
 3   SO2     float32
 4   CO_ug   float32
 5   Ozone   float32
 6   Temp    float32
 7   RH      float32
 8   WS      float32
 9   BP      float32
 10  WD      float32
dtypes: float32(11)
memory usage: 138.7 MB


In [22]:
df = df.interpolate(method='pad')
df = df.fillna(df.mean())

/var/folders/yp/jdzdnmvx69n8p5szmm_1rbhc0000gp/T/ipykernel_956/3343059986.py:1: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.interpolate(method='pad')


## Calcualte AQI

In [23]:
df.head()

,PM2_5,PM10,NO2,SO2,CO_ug,Ozone,Temp,RH,WS,BP,WD
0,142.699997,390.000000,17.920000,2.78,0.86922,39.240002,23.098722,38.610001,1.45,942.724731,243.399994
1,130.429993,326.850006,17.610001,2.69,0.86922,30.930000,23.098722,39.480000,1.89,942.724731,250.309998
2,114.449997,274.279999,24.290001,2.45,0.86922,26.150000,23.098722,41.330002,1.34,942.724731,237.669998
3,148.470001,292.000000,37.680000,2.50,0.86922,22.160000,23.098722,48.070000,1.45,942.724731,234.440002
4,151.220001,449.299988,46.610001,3.21,0.86922,21.950001,23.098722,51.320000,2.34,942.724731,238.669998


In [24]:
df['AQI (PM2.5)'] = np.nan

In [25]:
def calculate_aqi(df):
    # Calculate AQI for each pollutant using appropriate formulas
    df['AQI (PM2.5)'] = df['PM2_5'].apply(calculate_aqi_pm25)
    # aqi_pm10 = calculate_aqi_pm10(df['PM10'])
    # Calculate AQI for other pollutants similarly...

    # Determine overall AQI
    # overall_aqi = max(aqi_pm25, aqi_pm10, ...)  # Add AQI values for other pollutants

    # return aqi_pm25

# Example function to calculate AQI for PM2.5
def calculate_aqi_pm25(pm25):
    # AQI breakpoints and corresponding index values for PM2.5
    breakpoints = [0, 12, 35.4, 55.4, 150.4, 250.4, 350.4, 500.4]
    index_values = [0, 50, 100, 150, 200, 300, 400, 500]

    # Find the AQI category based on the concentration
    for i in range(len(breakpoints) - 1):
        if breakpoints[i] <= pm25 <= breakpoints[i + 1]:
            aqi_low = index_values[i]
            aqi_high = index_values[i + 1]
            breakpoint_low = breakpoints[i]
            breakpoint_high = breakpoints[i + 1]
            break
    else:
        # Handle out-of-range cases
        if pm25 < breakpoints[0]:
            aqi_low = index_values[0]
            aqi_high = index_values[0]
            breakpoint_low = breakpoints[0]
            breakpoint_high = breakpoints[1]
        elif pm25 > breakpoints[-1]:
            aqi_low = index_values[-1]
            aqi_high = index_values[-1]
            breakpoint_low = breakpoints[-2]
            breakpoint_high = breakpoints[-1]

    # Calculate AQI
    aqi = ((aqi_high - aqi_low) / (breakpoint_high - breakpoint_low)) * (pm25 - breakpoint_low) + aqi_low

    return aqi

# Calculate overall AQI for the DataFrame
calculate_aqi(df)
# print("Overall AQI:", overall_aqi)

In [26]:
df.head()

,PM2_5,PM10,NO2,SO2,CO_ug,Ozone,Temp,RH,WS,BP,WD,AQI (PM2.5)
0,142.699997,390.000000,17.920000,2.78,0.86922,39.240002,23.098722,38.610001,1.45,942.724731,243.399994,195.947367
1,130.429993,326.850006,17.610001,2.69,0.86922,30.930000,23.098722,39.480000,1.89,942.724731,250.309998,189.489470
2,114.449997,274.279999,24.290001,2.45,0.86922,26.150000,23.098722,41.330002,1.34,942.724731,237.669998,181.078946
3,148.470001,292.000000,37.680000,2.50,0.86922,22.160000,23.098722,48.070000,1.45,942.724731,234.440002,198.984211
4,151.220001,449.299988,46.610001,3.21,0.86922,21.950001,23.098722,51.320000,2.34,942.724731,238.669998,200.820001


In [27]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.1, random_state=42) 
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42) 

# Save Dataframes

In [30]:
train_df.to_csv('train_delhi_aqi.csv',index_label=False)
val_df.to_csv('val_delhi_aqi.csv',index_label=False)
test_df.to_csv('test_delhi_aqi.csv',index_label=False)